## Problem Description
***
Determine the optimal number of units to produce to maximize total revenue given the following constraints:
* At least 500 units of each product must be produced
* Hyperlinks must be >= 2*Speedlinks
* Production must be less than available raw materials

Available Materials

| PC Board (sq. in.) | Resistors | Memory Chips | Assembly Labor (hours) |  
|--------------------|-----------|--------------|------------------------|  
| 80,000             | 10,0000   | 30,000       | 5,000                  |


Costs

|                   |HyperLink  | FastLink	 | SpeedLink	| MicroLink	| EtherLink |
|-------------------|-----------|------------|--------------|-----------|-----------|
|Wholesale price    |	189	    |   149	     |      149	    |169	    | 139       |
|Manufacturing cost	|   136	    |   101	     |      96	    |137	    | 101       |

<b>Objective: </b> Minimize production * wholesale price - production * cost



In [4]:
import numpy as np 
import pandas as pd
import pulp

In [95]:
#fixed variables
materials_inv = [80000, 100000, 30000, 5000]
wholesale_price = [189, 149, 149, 169, 139]
manufacturing_cost = [136, 101, 96, 137, 101]

products = ['HyperLink', 'FastLink', 'SpeedLink', 'MicroLink', 'EtherLink']
materials = ['PC Board (sq.in.)', 'Resistors', 'Memory Chips', 'Assembly Labor (hours)']

min_production = 500

n_products = len(products)
n_materials = len(materials)

df_materials = pd.DataFrame([[20, 15, 10, 8, 5],
                            [22, 24, 18, 12, 16],
                            [6, 8, 4, 4, 6],
                            [0.75, 0.6, 0.5, 0.65, 1]], columns=products, index=materials)

In [96]:
#Create the model
model = pulp.LpProblem(name='spectra_optimization', sense=pulp.LpMaximize)

requirement_variables = pulp.LpVariable.matrix('P', products,
                                              lowBound=0, cat=pulp.LpInteger)


#The sum of each material must be less than or equal to the number of materials available
for index, m in enumerate(materials):
    model.addConstraint(pulp.LpConstraint(
        e=pulp.lpSum(requirement_variables*df_materials.loc[m,:].values) ,
        sense=pulp.LpConstraintLE,
        name=m + '_inv_Max',
        rhs=materials_inv[index]))

for index, p in enumerate(products):
    model.addConstraint(pulp.LpConstraint(
        e=requirement_variables[index] ,
        sense=pulp.LpConstraintGE,
        name=p + 'prod_Min',
        rhs=min_production))

model.addConstraint(pulp.LpConstraint(
    e=requirement_variables[products.index('HyperLink')]-2*requirement_variables[products.index('SpeedLink')],
    sense = pulp.LpConstraintGE,
    name = 'HyperLinkConstraint',
    rhs = 0))

objective = pulp.lpSum((np.array(wholesale_price)-np.array(manufacturing_cost))*requirement_variables)

model.setObjective(objective)

model.solve()

if model.status == 1:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')
    print(f'objective: ${model.objective.value():,.0f}')
    output = pd.DataFrame()
    for index,product in enumerate(products):
        output[product]=[requirement_variables[index].value()]
    
    print(output)
else:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')

status: 1, Optimal
objective: $251,346
   HyperLink  FastLink  SpeedLink  MicroLink  EtherLink
0     1572.0     500.0      786.0     2603.0      502.0


In [74]:
model

spectra_optimization:
MAXIMIZE
38.0*P_EtherLink + 48.0*P_FastLink + 53.0*P_HyperLink + 32.0*P_MicroLink + 53.0*P_SpeedLink + 0.0
SUBJECT TO
PC_Board_(sq.in.)_inv_Max: 5 P_EtherLink + 15 P_FastLink + 20 P_HyperLink
 + 8 P_MicroLink + 10 P_SpeedLink <= 80000

Resistors_inv_Max: 16 P_EtherLink + 24 P_FastLink + 22 P_HyperLink
 + 12 P_MicroLink + 18 P_SpeedLink <= 10000

Memory_Chips_inv_Max: 6 P_EtherLink + 8 P_FastLink + 6 P_HyperLink
 + 4 P_MicroLink + 4 P_SpeedLink <= 30000

Assembly_Labor_(hours)_inv_Max: P_EtherLink + 0.6 P_FastLink
 + 0.75 P_HyperLink + 0.65 P_MicroLink + 0.5 P_SpeedLink <= 5000

HyperLinkprod_Min: P_HyperLink >= 500

FastLinkprod_Min: P_FastLink >= 500

SpeedLinkprod_Min: P_SpeedLink >= 500

MicroLinkprod_Min: P_MicroLink >= 500

EtherLinkprod_Min: P_EtherLink >= 500

VARIABLES
P_EtherLink Continuous
P_FastLink Continuous
P_HyperLink Continuous
P_MicroLink Continuous
P_SpeedLink Continuous